In [ ]:
import os
import sys

# Add project root to Python path
project_root = os.path.abspath(os.path.join(os.getcwd(), ".."))
if project_root not in sys.path:
    sys.path.insert(0, project_root)


import ast

import numpy as np


from utils.gcloud_utilities import *

from utils.metadata import *

from utils.preprocessing_utilities import (
    import_operating_nodes,
    expand_parameters_col_and_format,
)

In [ ]:
year = "2023"

# Load data
bucket, nodes = import_operating_nodes(year)
endUse_nodes = pull_from_gcs_csv(
    bucket, GCLOUD_PREPROCESSED_DIR + BENCHMARK_PREPROCESSED_DIR + "endUse_nodes.csv"
)

edges = pull_from_gcs_csv(
    bucket,
    GCLOUD_PREPROCESSED_DIR
    + BENCHMARK_PREPROCESSED_DIR
    + BENCHMARK_EDGES_DIR
    + BENCHMARK_EDGES_FILE,
)

edges["properties"] = edges["properties"].astype(str).apply(ast.literal_eval)
dict_df = pd.json_normalize(edges["properties"])
edges = edges.drop(columns=["properties"]).join(dict_df)

nodes_df = pd.concat((nodes, endUse_nodes))

nodes_df["type"] = (
    nodes_df["mine_type"]
    .fillna(nodes_df["process_type"])
    .fillna(nodes_df["product_type"])
)
nodes_df["country"] = nodes_df["country"].fillna(nodes_df["region"])
nodes_df["company"] = nodes_df["company"].fillna(nodes_df["operator_short_clean"])

stages_dict = {
    "mining": ["Brine", "Spodumene", "Mica", "Pegmatite"],
    "carbonate": ["Lithium Carbonate"],
    "hydroxide": ["Lithium Hydroxide"],
    "cathode": [
        "NCM mid nickel",
        "LFP",
        "4V Ni or Mn based",
        "NCA",
        "NCM high nickel",
        "LCO",
        "NCM low nickel",
        "5V Mn based",
    ],
    "battery": [
        "Cylindrical",
        "Pouch",
        "Cylindrical, Pouch",
        "Pouch, Prismatic",
        "Prismatic",
        "Cylindrical, Prismatic",
        "Cylindrical, Pouch, Prismatic",
    ],
    "end_use": ["EV", "ESS", "Portable"],
}

nodes_df["stage"] = nodes_df["type"].map(
    {item: cat for cat, items in stages_dict.items() for item in items}
)

# nodes_df = nodes_df.dropna(subset=['stage'])

2025-05-10 16:33:30.849 | INFO     | utils.gcloud_utilities:fetch_gcs_bucket:16 - Fetching GCS bucket: lithium-datasets in project: critical-minerals'
c:\Users\bapti\OneDrive - University of Cambridge\Documents\critical-minerals\.venv\Lib\site-packages\google\auth\_default.py:76: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)
2025-05-10 16:33:33.577 | INFO     | utils.gcloud_utilities:pull_from_gcs_csv:27 - Pulling data from preprocessed/benchmark/benchmark_nodes.csv in bucket lithium-datasets
2025-05-10 16:33:35.338 | INFO     | utils.gcloud_utilities:pull_from_gcs_csv:27 - Pulling data from preprocessed/benchmark/endUse_nodes.csv in bucket lithium-datasets
2025-05-10 16:33:35.494

In [ ]:
inputs = edges.merge(
    nodes_df[["node_id", "type", "stage", "country", "company", "name"]],
    left_on=["source", "edge_type"],
    right_on=["node_id", "type"],
    how="left",
)
outputs = inputs[
    [
        "stage",
        "type",
        "source",
        "target",
        "2023_volume",
        "edge_type",
        "edge_destination",
        "country",
        "company",
        "name",
    ]
].merge(
    nodes_df[["node_id", "stage", "type", "country", "company", "name"]],
    left_on="target",
    right_on="node_id",
    how="left",
    suffixes=("_source", "_target"),
)
outputs = outputs[
    (outputs["edge_destination"].isna())
    | (outputs["edge_destination"] == outputs["type_target"])
]

In [4]:
final_node = 1469

In [ ]:
suppliers = outputs[outputs["target"] == final_node].sort_values(
    "2023_volume", ascending=False
)
total_buyer_flow = (
    suppliers.groupby(["edge_type"])
    .sum()
    .reset_index()[["edge_type", "2023_volume"]]
    .rename(columns={"2023_volume": "max_buyer_flow (t)"})
)
suppliers = suppliers.merge(total_buyer_flow, on="edge_type", how="left")

In [ ]:
uk_upstream_node = 631
uk_exp = outputs[outputs["source"] == uk_upstream_node]
uk_orig = uk_exp[[i in suppliers["source"].values for i in uk_exp["target"]]]
uk_orig = uk_orig[["target", "edge_destination", "2023_volume"]]

total_orig = (
    outputs[[i in suppliers["source"].values for i in outputs["target"]]]
    .groupby(["target", "edge_destination"])
    .sum()
    .reset_index()[["target", "edge_destination", "2023_volume"]]
)

uk_orig_perc = uk_orig.merge(total_orig, on=["target", "edge_destination"], how="left")
uk_orig_perc["perc_flow"] = (
    uk_orig_perc["2023_volume_x"] / uk_orig_perc["2023_volume_y"]
)

china_uk_flow = outputs[outputs["target"] == uk_upstream_node]
china_perc_imports = (
    china_uk_flow[china_uk_flow["country_source"] == "CHN"]["2023_volume"].sum()
    / china_uk_flow["2023_volume"].sum()
)

uk_orig_perc["CNUK_flow_proportion"] = uk_orig_perc["perc_flow"] * china_perc_imports
uk_orig_perc = uk_orig_perc[["target", "edge_destination", "CNUK_flow_proportion"]]

suppliers = suppliers.merge(
    uk_orig_perc,
    left_on=["source", "edge_type"],
    right_on=["target", "edge_destination"],
    how="left",
)
suppliers["CNUK_flow_proportion"] = suppliers["CNUK_flow_proportion"].fillna(0)
suppliers.drop(columns=["target_y", "edge_destination_y"], inplace=True)

In [ ]:
top_competitor = outputs[
    [i in suppliers["source"].values for i in outputs["source"]]
].sort_values(["source", "type_source", "2023_volume"], ascending=False)
top_competitor = top_competitor[top_competitor["target"] != final_node].drop_duplicates(
    subset=["source", "type_source"]
)
top_possible_competitors = (
    outputs[[i in top_competitor["target"].values for i in outputs["target"]]]
    .groupby(["target", "edge_type"])
    .sum()
    .reset_index()[["target", "edge_type", "2023_volume"]]
    .rename(columns={"2023_volume": "max_competitor_flow (t)"})
)
top_competitor = top_competitor.merge(
    top_possible_competitors, on=["target", "edge_type"], how="left"
)

In [ ]:
total_flow = (
    outputs[["source", "edge_type", "2023_volume"]][
        [i in suppliers["source"].values for i in outputs["source"]]
    ]
    .groupby(["source", "edge_type"])
    .sum()
    .reset_index()
)

In [ ]:
company_summary = suppliers.merge(
    top_competitor, on=["source", "edge_type"], how="left"
)
company_summary = company_summary.merge(
    total_flow, on=["source", "edge_type"], how="left"
)

company_summary = company_summary[
    [
        "stage_source_x",
        "type_source_x",
        "country_source_x",
        "company_source_x",
        "name_source_x",
        "2023_volume_x",
        "max_buyer_flow (t)",
        "CNUK_flow_proportion",
        "country_target_y",
        "company_target_y",
        "name_target_y",
        "2023_volume_y",
        "max_competitor_flow (t)",
        "2023_volume",
    ]
].rename(
    columns={
        "stage_source_x": "supplier_stage",
        "type_source_x": "material_type",
        "country_source_x": "supplier_country",
        "company_source_x": "supplier_company",
        "name_source_x": "supplier_facilityName",
        "2023_volume_x": "product_flow (t)",
        "country_target_y": "competitor_country",
        "company_target_y": "competitor_company",
        "name_target_y": "competitor_facilityName",
        "2023_volume_y": "competitor_flow (t)",
        "2023_volume": "total_supplier_flow (t)",
    }
)

company_summary["max_competitor_flow (t)"] = np.minimum(
    company_summary["max_competitor_flow (t)"],
    company_summary["total_supplier_flow (t)"],
)
company_summary["max_buyer_flow (t)"] = np.minimum(
    company_summary["max_buyer_flow (t)"], company_summary["total_supplier_flow (t)"]
)

In [ ]:
# company_summary.to_csv('/Users/lukecullen/Library/CloudStorage/OneDrive-SharedLibraries-UniversityofCambridge/ENG_CCG - Y5/Non-ODA/P4 UK-China/supplier_data/Envision_AESC_Sunderland_Pouch_batteries_supplier_data.csv', index=False)